In [3]:
from langchain_community.document_loaders import TextLoader

from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


In [2]:
# from dotenv import load_dotenv
# import os
#
# load_dotenv()
# print("OPENAI_API_KEY: " +os.getenv("OPENAI_API_KEY") + "\n")
# print("HUGGINGFACE_TOKEN: "+os.getenv("HUGGINGFACE_TOKEN") + "\n")
# print(os.getcwd())

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import pandas as pd
books = pd.read_csv("book_cleaned.csv")
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [6]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [7]:
books["tagged_description"].to_csv("tagged_description.txt",
                                  sep = "\t",
                                  index = False,
                                  header = False)

In [8]:
from langchain.schema import Document

raw_documents = TextLoader("tagged_description.txt", encoding = "utf-8-sig").load()
#text_splitter = CharacterTextSplitter(chunk_size = 0, chunk_overlap = 0, separator = "\n" )
# Lấy nội dung văn bản gốc
text = raw_documents[0].page_content

# Tách theo dòng
lines = [line.strip() for line in text.split("\n") if line.strip()]

# Tạo danh sách Document
documents = [Document(page_content=line) for line in lines]

print(f"✅ Created {len(documents)} documents (1 per line)")

print(f"✅ Loaded {len(documents)} document(s)")

✅ Created 5197 documents (1 per line)
✅ Loaded 5197 document(s)


In [9]:
documents[0]

Document(metadata={}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and 

In [10]:
import os
print(os.path.exists("tagged_description.txt"))

True


In [11]:

api_key = os.getenv("OPENAI_API_KEY")
project_id = os.getenv("OPENAI_PROJECT_ID")

embedding = OpenAIEmbeddings(
    api_key = api_key,
)

db_books = Chroma.from_documents(
    documents,
    embedding = embedding
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


C:\Users\Acer\AppData\Local\Temp\ipykernel_24936\2116722447.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

D:\PyCharm Projects\book-recommender\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Acer\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [77]:
from dotenv import load_dotenv
import os

# Load file .env từ thư mục hiện tại
env_path = os.path.join(os.getcwd(), ".env")
print("Loading .env from:", env_path)

load_dotenv(dotenv_path=env_path)

# Kiểm tra lại
print("API key:", os.getenv("OPENAI_API_KEY"))


Loading .env from: D:\PyCharm Projects\book-recommender\.env
API key: sk-proj-qo6TY-d7RkMcqOXC-uWYtIcUc5BhIYsk-...blIAIRH48mLjU2QSyetg2CLYuS6-8cEAHEmzPJ4JUm4So0j_T3BlbkFJMvLb2xhaRBzU1O-FOh5e-5x8uUqdN7WM7KQ3jUz6uKgq5NFY6d9jZ2b8z496LFkqsbdK1XblIA


In [22]:
from dotenv import load_dotenv
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# Load biến môi trường
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
project_id = os.getenv("OPENAI_PROJECT_ID")

embedding = OpenAIEmbeddings(
    api_key=api_key,
    model="text-embedding-3-small"
)

db_books = Chroma.from_documents(
    documents,
    embedding=embedding
)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************1S4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [12]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

# Đọc file .env
load_dotenv()

# Lấy token
hf_token = os.getenv("HUGGINGFACE_TOKEN")

# Đăng nhập vào Hugging Face
login(hf_token)

print("Đăng nhập Hugging Face thành công!")


Đăng nhập Hugging Face thành công!


In [15]:
db_books = Chroma.from_documents(documents, embedding=embedding)

In [16]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs


[Document(id='f79436e3-16fd-44e0-887f-d24a98bb0100', metadata={}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='b2749f87-164e-4a34-bad3-5605ab48258b', metadata={}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='c9582a6d-ebec-44f4-9f2f-145f32b556bc', metadata={}, page_content="9780786808397 Introduce your baby to birds, cats, dogs, and ba

In [17]:
for r in docs:
    print(r.page_content)

9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.
9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects.
9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to exopse little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies eac

In [18]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [19]:
def retrieve_semantic_recommendations(query: str, top_k: int = 10, ) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k =50)

    books_list =[]
    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)

In [22]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240 It should be the most natural th...
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
334,9780060976118,006097611X,Operation Wandering Soul,Richard Powers,Fiction,http://books.google.com/books/content?id=nIGIm...,"Highly imaginative and emotionally powerful, t...",1994.0,3.62,352.0,366.0,Operation Wandering Soul,9780060976118 Highly imaginative and emotional...
383,9780061144899,0061144894,When the Heart Waits,Sue Monk Kidd,Religion,http://books.google.com/books/content?id=JlP91...,From the Bestselling Author of The Secret Life...,2006.0,4.17,240.0,2141.0,When the Heart Waits: Spiritual Direction for ...,9780061144899 From the Bestselling Author of T...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
406,9780064403870,0064403874,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=WTHHH...,"When Margaret and her younger brother, Artie, ...",1991.0,3.52,272.0,631.0,"R-T, Margaret, and the Rats of NIMH",9780064403870 When Margaret and her younger br...
407,9780064404419,0064404412,The Rainbow People,Laurence Yep,Juvenile Fiction,http://books.google.com/books/content?id=5AHwq...,"""Culled from 69 stories collected in a [1930s]...",1992.0,3.75,208.0,202.0,The Rainbow People,"9780064404419 ""Culled from 69 stories collecte..."
416,9780064406925,006440692X,Winter on the Farm,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=IvlKH...,The Little House books tell the story of a lit...,1997.0,4.13,32.0,400.0,Winter on the Farm,9780064406925 The Little House books tell the ...
427,9780064434874,0064434877,Christmas in the Big Woods,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=FT1Yp...,"Long ago, a little girl named Laura Ingalls li...",1997.0,4.19,32.0,2062.0,Christmas in the Big Woods,"9780064434874 Long ago, a little girl named La..."
429,9780064434980,0064434982,The Deer in the Wood,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=V7YDW...,Even the youngest child can enjoy a special ad...,1999.0,4.17,32.0,302.0,The Deer in the Wood,9780064434980 Even the youngest child can enjo...


In [23]:
retrieve_semantic_recommendations("Doraemon")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
77,9780020264736,0020264739,The Song of Rhiannon,Evangeline Walton,Fiction,NaN,A retelling of The Mabinogion in novel form. M...,1992.0,4.02,208.0,247.0,The Song of Rhiannon: The Third Branch of the ...,9780020264736 A retelling of The Mabinogion in...
459,9780099271499,0099271494,Blood and Gold,Anne Rice,Fiction,http://books.google.com/books/content?id=kq8A8...,Here is the gorgeous and sinister story of Mar...,2002.0,3.89,752.0,27476.0,Blood and Gold: The Vampire Marius,9780099271499 Here is the gorgeous and siniste...
687,9780140447262,0140447261,The Bacchae and Other Plays,Euripides;John N. Davie;R. B. Rutherford,Drama,http://books.google.com/books/content?id=mWhUe...,"Through their sheer range, daring innovation, ...",2005.0,4.13,360.0,153.0,The Bacchae and Other Plays,"9780140447262 Through their sheer range, darin..."
1004,9780195145663,0195145666,Medea,Eurípides;Michael Collier;Georgia Machemer,Drama,http://books.google.com/books/content?id=9mViA...,"Medea, whose magical powers helped Jason and t...",2006.0,3.85,116.0,339.0,Medea,"9780195145663 Medea, whose magical powers help..."
1047,9780226307800,0226307808,Euripides I,Euripides,Greek drama (Tragedy),http://books.google.com/books/content?id=ZRt5I...,Table of Contents: Alcestis / translated by Ri...,1955.0,4.03,221.0,4482.0,Euripides I: Alcestis; The Medea; The Heraclei...,9780226307800 Table of Contents: Alcestis / tr...
1242,9780312357771,031235777X,The Silver Pigs,Lindsey Davis,Fiction,http://books.google.com/books/content?id=a0wmu...,"When Marcus Didius Falco, a Roman ""informer"" w...",2006.0,3.94,329.0,8592.0,The Silver Pigs: A Marcus Didius Falco Mystery,"9780312357771 When Marcus Didius Falco, a Roma..."
1455,9780345396709,0345396707,Waterborn,J. Gregory Keyes,Fiction,http://books.google.com/books/content?id=JUgi-...,"When her favorite cousin disappears, Hezhi, an...",1997.0,3.80,384.0,74.0,Waterborn,9780345396709 When her favorite cousin disappe...
1546,9780345477972,0345477979,Tsubasa,CLAMP (Mangaka group),Juvenile Fiction,http://books.google.com/books/content?id=kuh6t...,"Sakura, the princess of Clow, has a strange po...",2005.0,4.29,194.0,3392.0,Tsubasa: vol. 7,"9780345477972 Sakura, the princess of Clow, ha..."
1562,9780345484291,0345484290,Tsubasa 9 Reservoir Chronicle,CLAMP (Mangaka group),NaN,NaN,"Sakura, the princess of Clow, has a strange po...",2004.0,4.28,184.0,3005.0,Tsubasa 9 Reservoir Chronicle: In the Gods' Hands,"9780345484291 Sakura, the princess of Clow, ha..."
1564,9780345485281,0345485289,"Tsubasa, 11",CLAMP (Mangaka group),Amnesia,NaN,"Sakura, the princess of Clow, has a strange po...",2004.0,4.28,183.0,2674.0,"Tsubasa, 11: Reservoir Chronicle (YA)","9780345485281 Sakura, the princess of Clow, ha..."


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
12,9780006486145,0006486142,Well of Darkness,Margaret Weis;Tracy Hickman,NaN,http://books.google.com/books/content?id=XrwaA...,Gareth is just a frightened young lad when he ...,2001.0,3.66,599.0,68.0,Well of Darkness,9780006486145 Gareth is just a frightened youn...
146,9780060556501,0060556501,"The Lion, the Witch and the Wardrobe (picture ...",C. S. Lewis,Juvenile Fiction,http://books.google.com/books/content?id=FlSpp...,Narnia: A magical land full of wonder and exci...,2004.0,4.19,48.0,5012.0,"The Lion, the Witch and the Wardrobe (picture ...",9780060556501 Narnia: A magical land full of w...
220,9780060765453,0060765453,The Chronicles of Narnia Movie Tie-in Edition ...,C. S. Lewis,Fiction,http://books.google.com/books/content?id=4qVxv...,When Digory and Polly try to return the wicked...,2005.0,4.26,766.0,1099.0,The Chronicles of Narnia Movie Tie-in Edition ...,9780060765453 When Digory and Polly try to ret...
346,9780061020704,0061020702,The Light Fantastic,Terry Pratchett,Fiction,http://books.google.com/books/content?id=fAU6u...,Terry Pratchett's profoundly irreverent novels...,2000.0,3.97,241.0,86465.0,The Light Fantastic,9780061020704 Terry Pratchett's profoundly irr...
440,9780066238500,0066238501,The Chronicles of Narnia (adult),C. S. Lewis,Fiction,http://books.google.com/books/content?id=3VGkK...,"Journeys to the end of the world, fantastic cr...",2001.0,4.26,767.0,425445.0,The Chronicles of Narnia (adult),9780066238500 Journeys to the end of the world...
501,9780099456452,0099456451,The Spook's Apprentice,Joseph Delaney,Exorcism,http://books.google.com/books/content?id=ZA0yS...,A wonderful and terrifying series by a new wri...,2005.0,3.98,325.0,2733.0,The Spook's Apprentice,9780099456452 A wonderful and terrifying serie...
744,9780141183350,0141183357,Howards End,Edward Morgan Forster,Brothers,http://books.google.com/books/content?id=s6chP...,This novel by the author of Maurice and A Pass...,2000.0,3.96,352.0,367.0,Howards End,9780141183350 This novel by the author of Maur...
779,9780141311883,0141311886,The Blue Sword,Robin McKinley,NaN,http://books.google.com/books/content?id=nDW2H...,"Harry, bored with her sheltered life in the re...",2001.0,4.23,272.0,286.0,The Blue Sword,"9780141311883 Harry, bored with her sheltered ..."
917,9780156032971,015603297X,Foucault's Pendulum,Umberto Eco,Fiction,http://books.google.com/books/content?id=Ubkvm...,"Three editors, inspired by an extraordinary fa...",2007.0,3.90,623.0,44658.0,Foucault's Pendulum,"9780156032971 Three editors, inspired by an ex..."
1247,9780312369347,0312369344,Beatrix Potter,Linda Lear,Biography & Autobiography,http://books.google.com/books/content?id=T9uTx...,"Peter Rabbit, Mr. McGregor, and many other Bea...",2007.0,4.03,584.0,1270.0,Beatrix Potter: A Life in Nature,"9780312369347 Peter Rabbit, Mr. McGregor, and ..."
